# Top Movies Data Processing

This notebook contains steps to aggregate [top_movies_data.csv](https://github.com/the-pudding/data/tree/master/film-or-digital/top_movies_data.csv) to the ones visualized in The Pudding essay [Film or Digital: Breaking Down Hollywood's Choice of Shooting Medium](https://pudding.cool/2018/08/film-or-digital/).

In [ ]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from IPython.core.display import display

In [ ]:
#load data
dataset = pd.read_csv("top_movies_data.csv")

In [ ]:
#exclude genre Animation and Documentary
dataset=dataset[~dataset['genres'].str.contains("Animation|Documentary")]

#exclude movies with unknown medium
dataset=dataset[~dataset["film_type"].str.contains("U")]

In [ ]:
#expand film type to new columns
dummies=pd.get_dummies(dataset,columns=["film_type"])
dataset=pd.concat([dataset,dummies[["film_type_D","film_type_D|F","film_type_F"]]],axis=1)

------
## Prepare some functions to map values/formatting later

In [ ]:
#Function to calculate percentage of film/digital/both per aggregation group.
def get_percentage(series):
    sumtotal=series.sum()
    return series/sumtotal*100

In [ ]:
#Function to round percentage breakdown using Largest Remainder Method
#useful when making waffle chart, to make sure the percentage breakdown add up to exactly 100 percent.
#https://stackoverflow.com/questions/13483430/how-to-make-rounded-percentages-add-up-to-100

def largest_remainder(series):
    series_pct=get_percentage(series) #calculate percentage breakdown
    floored_series=series_pct.apply(np.floor)
    remainder_series=series_pct-floored_series
    sumtotal=floored_series.sum()
    remainder=100-sumtotal
    for index,values in remainder_series.sort_values(ascending=False).iteritems():
        if remainder>0:
            floored_series[index]+=1
            remainder-=1
    return floored_series

In [ ]:
#Function to map year to a three-years period.
def calculate_period(series):
    #Categorize production year to period.
    #period 1: 2006-2008, period 2: 2009-2012, etc

    condition=[
                series.between(2006,2008),
                series.between(2009,2011),
                series.between(2012,2014),
                series.between(2015,2017),
              ]

    category=[
                "2006 - 2008", "2009 - 2011", "2012 - 2014", "2015 - 2017"
             ]
    return np.select(condition,category)

------
# Compute Mediums of Top Movies by Year

In [ ]:
#select relevant columns for this analysis
movie_list=dataset[["production_year","title","film_type_D","film_type_D|F","film_type_F"]]

In [ ]:
#group movies medium per year
movie_medium_peryear=movie_list.groupby("production_year").sum()
movie_medium_peryear.columns=["Digital","Both","Film"]
movie_medium_peryear["Total"]=movie_medium_peryear["Digital"]+movie_medium_peryear["Both"]+movie_medium_peryear["Film"]

In [ ]:
#round result using Largest Remainder Method
finalresult=movie_medium_peryear[["Digital","Both","Film"]].apply(lambda x:largest_remainder(x), axis=1)

In [ ]:
#display result 
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', -1):
    display(finalresult)

------
# Compute Medium of Top Movies by Genre and Period

In [ ]:
#select relevant columns for this analysis
movie_list=dataset[["production_year","title","genres","film_type_D","film_type_D|F","film_type_F"]]

In [ ]:
#Map production year to period.
movie_list["period"]=movie_list[["production_year"]].apply(calculate_period)
movie_list.drop(["production_year"],axis=1,inplace=True)

In [ ]:
#compute stats for all genres
movie_medium_allgenre=movie_list.groupby(["period"]).sum()
movie_medium_allgenre.columns=["Digital","Both","Film"]
movie_medium_allgenre=pd.concat([movie_medium_allgenre], keys=['All'], names=['Genres'])

In [ ]:
#split movie genres in multiple rows, to use in group by method later
genre_split = movie_list['genres'].str.split('|').apply(pd.Series, 1).stack()
genre_split.index=genre_split.index.droplevel(-1)
genre_split.name = 'genres_split'
movie_list=movie_list.join(genre_split)

In [ ]:
#compute stats per genre
movie_medium_bygenre=movie_list.groupby(["genres_split","period"]).sum()
movie_medium_bygenre.columns=["Digital","Both","Film"]

# and combine them with stats across all genres
movie_medium_bygenre=pd.concat([movie_medium_allgenre,movie_medium_bygenre])

In [ ]:
#round result using Largest Remainder Method
finalresult=movie_medium_bygenre.apply(lambda x:largest_remainder(x), axis=1)

In [ ]:
#display result 
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', -1):
    display(finalresult)

------
# Medium of Top Movies, by Budget Range and Period

In [ ]:
#select relevant columns for this analysis
movie_list=dataset[["production_year","title","budget","film_type_D","film_type_D|F","film_type_F"]]

In [ ]:
#Categorize production year to period
movie_list["period"]=movie_list[["production_year"]].apply(calculate_period)

#and drop production year column
movie_list.drop(["production_year"],axis=1,inplace=True)

In [ ]:
#drop movies with unknown budget
movie_list=movie_list[movie_list["budget"]>0]

#Map budget to budget range
condition=[
            movie_list['budget']<20000000, 
            movie_list['budget'].between(20000000, 39999999),
            movie_list['budget'].between(40000000, 79999999),
            movie_list['budget'].between(80000000, 159999999),
            movie_list['budget']>=160000000
          ]

category=[
            "a. <20M","b. 20M-40M","c. 40M-80M","d. 80M-160M","e. >=160M"
         ]

movie_list["budget_range"]=np.select(condition,category)

#and drop budget column
movie_list.drop(["budget"],axis=1,inplace=True)

In [ ]:
#group movie medium per period and budget range
movie_medium_bybudget=movie_list.groupby(["period","budget_range"]).sum()
movie_medium_bybudget.columns=["Digital","Both","Film"]

In [ ]:
#round result using Largest Remainder Method
tempresult=movie_medium_bybudget.apply(lambda x:largest_remainder(x), axis=1)

In [ ]:
#transform the table to the ones shown in the essay
finalresult=pd.DataFrame()
for medium in ["Digital","Film","Both"]:
    for period in movie_list["period"].unique():
        finalresult=finalresult.append(tempresult[[medium]].transpose()[period].set_index([[period]],append=True))
finalresult.index.set_names(["medium","period"],inplace=True)

In [ ]:
#display result 
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', -1):
    display(finalresult)

------
# Medium of Top Movies, by Budget and Period

In [ ]:
#select relevant columns for this analysis
movie_list=dataset[["id","production_year","title","budget","film_type"]]

In [ ]:
#Categorize production year to period
movie_list["period"]=movie_list[["production_year"]].apply(calculate_period)

In [ ]:
#drop movies with unknown budget
movie_list=movie_list[movie_list["budget"]>0]

In [ ]:
#rearrange data
movie_list.sort_values(by=["period","production_year","budget"],inplace=True)
movie_list.set_index("period",inplace=True)

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(movie_list)